In [2]:
# import matplotlib as mpl
from matplotlib import pyplot as plt
# from pyart import graph as pag
# import numpy as np
# from glob import glob
from datetime import datetime as dt
# import os
from tqdm import tqdm
# import sys
# import xarray as xr
# import cartopy.crs as ccrs
# import cartopy.feature as cf
# from pyproj import Proj as Proj
# from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from pyMRMS import retrieveMRMS,readFL,annotate,graph

plt.ioff()
# %matplotlib inline

In [8]:
projID = 'VSE'

getMRMS = False
strtMRMS='20170430-180000'
endMRMS='20170430-200000'
mrmsSaveDir='/Users/danstechman/Desktop/MRMS/'

pltFT = False
flFile = '/Users/danstechman/GoogleDrive/VSE-Data/flightLevel/20170430H1_AC.nc'
strtFL = strtMRMS
endFL = endMRMS

minLat = 33.5
minLon = -89
maxLat = 39.5
maxLon = -84



if getMRMS:
    retrieveMRMS.fetch(strtMRMS,endMRMS,mrmsSaveDir)

if pltFT:
    flData = readFL.getP3(flFile,strtDT=strtFL,endDT=endFL)

mData = retrieveMRMS.extract('/Users/danstechman/Desktop/MRMS/',strtDT=strtMRMS,endDT=endMRMS,llCrds=(minLat,minLon),urCrds=(maxLat,maxLon))

In [9]:
for t in tqdm(range(len(mData['mDT'])),dynamic_ncols=True):
    # print('Now plotting {}... {}/{}'.format(dt.strftime(dt.utcfromtimestamp(int((mData['mDT'][t].astype(dt))*1e-9)),'%Y%m%d-%H%M'),t+1,len(mData['mDT'])))
    fig,ax,grd,proj = graph.plotSHSR(mData['mLon'],mData['mLat'],mData['mSHSR'][t],mData['mDT'][t],projID='VSE',savePath=mrmsSaveDir)
    plt.close(fig)

100%|██████████| 61/61 [01:31<00:00,  1.51s/it]
